# AWSを使ってセンサーデータの可視化をしてみよう！

## IoTデバイスの普及

近年、IoTの成長は凄まじいです。

下のグラフは平成30年の情報通信白書に掲載されているIoTデバイスの推移となります。

統計によると、2020年には403億個のデバイスが世界とつながるのです！

[出典: 総務省 平成30年情報通信白書](http://www.soumu.go.jp/johotsusintokei/whitepaper/ja/h30/html/nd111200.html)

![graph-iot](http://www.soumu.go.jp/johotsusintokei/whitepaper/ja/h30/image/n1102010.png)

## 大量のデバイスのデータはいかがする？

400億以上のデバイスからくる大量のデータ。

一つ一つのデータ量は小さくても量が多くなればそれなりに工夫しないと分析は難しいでしょう。

大量のデータを収集し、必要なデータを抽出して分析しやすいような形に加工してし、データを貯めて、必要に応じて取り出し、可視化や機械学習のような高度な分析や予測をする必要があります。

本教材では、AWSのマネージド・サービスを使って簡単なETL処理基盤を作り、データの可視化をするというものです。

### 大量データの収集

IoTコースで学びます。

### 必要なデータを抽出し、分析しやすいように加工

IoTデバイスからくるデータをそのまま使うことはあまりおすすめしません。なぜでしょう。

IoTデバイスからくるデータはすべて同じ構造でくるとは限りません。欠損もあるかもしれません。社内の別のデータベースと対応させないと意味のないデータかもしれません。様々なデバイスから非同期でくるデータを1つにくっつけて、社内にある巨大なDBやオブジェクトストレージにインサートしてあげる必要もありそうです。

そこで、 `ETL` ということをする必要があります。

ETLとは、 *Extract/Transform/Load* の略で、データの抽出、データの変換加工、データウェアハウスへのロードという意味です。

本講習では、 AWS Glueというサービスを使います。

### データを貯める/データを取り出す

データ量が多くなるとハードディスクを圧迫し、しんどいのでできればサマった形で加工しておきたいものです。

しかしながらせっかく保存したデータも、月平均などでサマってしまうといざ特定の日のデータが使いたくなっても二度と使えなくなってしまいます。

近年は、ハードディスク単価が安くなったことで、 `データウェアハウス` という考えが主流となりました。

すなわち、時系列データをサマらずそのままの状態で保存することで、将来の意思決定に活用する、というものです。

また、データウェアハウスに保存したデータをSQLのように検索できると取り出しが楽です。

多くのデータウェアハウスはデータの検索やマイニングが容易な環境が用意されています。

本講習ではAWS S3、Athenaというサービスを利用します。

### データを可視化する

せっかくデータウェアハウスに入れたデータを使わない手はありません。

さっそく可視化してみましょう。

本講習では、Pythonの可視化ツール　Dash を使ってインタラクティブでインテリジェンスな可視化をしてできるビジネスパーソンを演じてみます。

また、 Elasticsearch ServiceというAWSマネージドサービスを使ってKibanaという可視化ツールを使うこともエキストラ講習で行います。


## 構成図

下に書いたものをAWS上で作ります。

![architecture](https://raw.githubusercontent.com/tubone24/rr-weather-data-with-aws/master/docs/images/architect.png)

## アウトプットイメージ

下記のような可視化画面ができあがれば完成です。

ボタンやマウスオーバで各地の詳細天気が出てきたりします。

![map2](https://raw.githubusercontent.com/tubone24/rr-weather-data-with-aws/master/docs/images/map2.gif)

## データセットのダウンロード

今回利用するデータセットをダウンロードします。

[Kaggle](http://www.soumu.go.jp/johotsusintokei/whitepaper/ja/h30/html/nd111200.html)というData Scientistな人たちが、分析結果を 競争するデータ分析プラットホームがあります。今回はそこからデータセットを拝借します。

予めKaggleのAPIが使えるようにユーザ登録を済ませましょう。

https://www.kaggle.com/account/login

ユーザ登録を済ませたら、下記サイト手順でAPIを使うためのクレデンシャルを準備します。

こちらの記事が参考になります。

[kaggle-api kaggleコマンドの使い方](https://qiita.com/uni-3/items/f1fdbeeddd08ca22c80f)

準備はできましたか？

今回ダウンロードするデータセットは[Weather Data for Recruit Restaurant Competition](https://www.kaggle.com/huntermcgushion/rrv-weather-data)です。

日本の1663個のデータステーションから取得できた気象データを扱います。

それではデータセットをダウンロードしましょう。
下記のコマンドを打ちます。

```
make download-dataset
```

そのまま、データ収集用のS3にデータをアップロードします。

先にデータを入れるS3バケットを作成します。

下記コマンドはTerraformというインフラコードツールです。

Terraformの解説はここではしませんが、かなり便利ですのでおすすめです。

```
make remote-enable ENV=aws-training COMPONENT=origin-datas
make create-env ENV=aws-training COMPONENT=origin-datas
make plan ENV=aws-training COMPONENT=origin-datas
> xx add xx changes xx destroy とか出てくる
make apply ENV=aws-training COMPONENT=origin-datas
```

S3バケットができたらデータを投入します。

```
make upload-weather-data
```

データをアップロードしたら、少し中身を見てみましょう。

お手元のエクセル、メモ帳(Atomなど)、またはJupyterで見ます。

In [5]:
import pandas as pd

df = pd.read_csv("../scripts/tmp_dir/1-1-16_5-31-17_Weather/akita__NONE__uno-tay.csv")
df.head()

,calendar_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
0,2016-01-01,-2.0,0.2,-4.5,2.5,0.0,NaN,41,6,0.5,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,1.2,5.2,-2.1,10.0,0.0,NaN,36,0,0.9,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,1.2,3.8,0.3,6.0,0.0,NaN,18,0,0.2,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,1.3,6.9,-0.2,9.5,0.0,NaN,13,0,0.8,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,-1.3,-0.1,-2.3,7.0,0.0,NaN,33,23,0.7,NaN,NaN,NaN,NaN,NaN


weatherデータは日次でデータが入っているようです。

平均気温の他、最高気温、最低気温、降水量、日照量などが入ってます。

詳しい解説は[Weather Data for Recruit Restaurant Competition](https://www.kaggle.com/huntermcgushion/rrv-weather-data))を見ましょう。

NaNというデータが入っていない項目もある程度あるようです。

stationの場所に関する情報は入っていませんでした。station_idも項目がありませんが、ファイル名がstation_idとなっているようです。

もう少し見てみましょう。

In [6]:
df.describe()

,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
count,517.000000,517.000000,517.000000,517.000000,517.000000,0.0,517.000000,517.000000,504.000000,0.0,0.0,0.0,0.0,0.0
mean,7.954932,12.182205,4.118569,5.273694,3.225919,NaN,30.646035,2.630561,1.291667,NaN,NaN,NaN,NaN,NaN
std,8.948734,10.065541,8.367684,10.155002,3.644150,NaN,44.246452,6.213332,0.694833,NaN,NaN,NaN,NaN,NaN
min,-7.400000,-5.500000,-11.400000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,-0.100000,2.600000,-2.900000,0.000000,0.100000,NaN,0.000000,0.000000,0.800000,NaN,NaN,NaN,NaN,NaN
50%,6.300000,11.200000,1.900000,1.500000,1.700000,NaN,0.000000,0.000000,1.150000,NaN,NaN,NaN,NaN,NaN
75%,16.300000,21.400000,11.200000,6.000000,5.900000,NaN,66.000000,2.000000,1.700000,NaN,NaN,NaN,NaN,NaN
max,26.300000,32.100000,21.300000,115.000000,12.000000,NaN,139.000000,57.000000,4.000000,NaN,NaN,NaN,NaN,NaN


このステーションではデータ数が517あったみたいです。

平均気温の平均は 7.95度だったようです。

この後AWSで分析しますので、いったんここまででやめときましょうか・

stationのデータも見てみましょう。

In [8]:
df2 = pd.read_csv("../scripts/tmp_dir/weather_stations.csv")
df2.head()

,id,prefecture,first_name,second_name,latitude,longitude,altitude,date_terminated
0,aichi__ai-xi-kana__isaai,aichi,ai-xi-kana,isaai,35.216667,136.698333,5.0,NaN
1,aichi__aki-kana__azo,aichi,aki-kana,azo,35.078333,137.413333,613.0,NaN
2,aichi__centrair-kana__centrair,aichi,centrair-kana,centrair,34.858333,136.805000,4.0,NaN
3,aichi__chapel-mountain-kana__chaw-yama,aichi,chapel-mountain-kana,chaw-yama,35.220000,137.660000,1216.0,NaN
4,aichi__gamagori-kana__gamagori,aichi,gamagori-kana,gamagori,34.845000,137.216667,55.0,NaN


こちらには、ステーションの情報があります。

idはステーションのIDですし、latitude、longitudeは緯度経度情報です。これは使えそう。

## ではさっそく分析してみましょう！

さあAWSを使ってワクワクな可視化をして行きましょう！